In [1]:
import pybullet as p
import pybullet_data as pd
import math
import time
import numpy as np
import sympy as sp
from numpy import sin, cos, pi
import panda as panda_sim


In [2]:
p.connect(p.GUI)
p.configureDebugVisualizer(lightPosition= (0,0,10))
p.setAdditionalSearchPath(r'C:\Users\vase_\Downloads\robot-simulation')
p.resetDebugVisualizerCamera( cameraDistance=15, cameraYaw=15, cameraPitch=-20, cameraTargetPosition=[0,0,0])


In [3]:
orn=[0, 0.0, 0.0, 1]
flags = p.URDF_ENABLE_CACHED_GRAPHICS_SHAPES
p.loadURDF("outfile.urdf", np.array([0,0,0]), orn, useFixedBase=True, flags=flags)

0

In [4]:
parameters = []
joints = []
for i in range(p.getNumJoints(0)):
    joint_info = p.getJointInfo(0,i)
    if joint_info[2] == p.JOINT_PRISMATIC:
        joints.append(i)
        p.resetJointState(0,i,0)
        parameters.append((False,p.addUserDebugParameter("D"+str(i),joint_info[8],joint_info[9],1)))
    if joint_info[2] == p.JOINT_REVOLUTE:
        joints.append(i)
        p.resetJointState(0,i,0)
        parameters.append((True,p.addUserDebugParameter("Theta"+str(i),np.rad2deg(joint_info[8]),np.rad2deg(joint_info[9]),0)))

In [5]:
file1 = open('outfile.urdf', 'r')
Lines = file1.readlines()
  
count = 0
DH_Params = []
# Strips the newline character
for line in Lines[1:]:
    if line.strip() == '-->':
        break
    dhlist = line.strip().split(',')
    DH_Params.append([dhlist[0]]+[float(param) for param in dhlist[1:]])
print(DH_Params)
    

[['r', 2.0, 0.0, 0.0], ['p', 0.0, 0.0, -1.5707963267948966], ['p', 0.0, 0.0, 0.0], ['r', 0.0, 0.0, 1.5707963267948966], ['r', 0.0, 0.0, -1.5707963267948966], ['r', 0.0, 2.0, 0.0]]


In [6]:
def DH_trans(DH, joint_val=0):

    d, theta, a, alpha = (0,0,0,0)

    if (DH[0] == 'r'):

        d, theta, a, alpha = (DH[1], joint_val, DH[2], DH[3])

    elif (DH[0] == 'p'):

        d, theta, a, alpha = (joint_val, DH[1], DH[2], DH[3])

    elif (DH[0] == 'f'):

        d, theta, a, alpha = (DH[1], DH[2], DH[3], DH[4])

    trans_mat = np.array([[cos(theta), -1*sin(theta)*cos(alpha), sin(theta)*sin(alpha),    a*cos(theta)],
                          [sin(theta), cos(theta)*cos(alpha),    -1*cos(theta)*sin(alpha), a*sin(theta)],
                          [0,          sin(alpha),               cos(alpha),               d           ],
                          [0,          0,                        0,                        1           ]])

    return trans_mat


def joint_transforms(DH_Params):

    transforms = []

    current_DOF = 0

    transforms.append(np.eye(4))
    i=0
    for DH in DH_Params:
        if(DH[0] != 'f'):
            transforms.append(DH_trans(DH, jointpos[i][0]))
            i+=1
        else:
            transforms.append(DH_trans(DH))

    return transforms

def joint_frames(transforms):
         
         joint_frames = [transforms[0]]
 
         for trans in transforms[1:]:
 
             joint_frames.append(joint_frames[-1] @ trans)
 
         return joint_frames
def axis_lines(pose,line_length=1):
    t = pose[:3, -1]
    line_n = [t, t + line_length*pose[:3,0]]
    line_o = [t, t + line_length*pose[:3,1]]
    line_a = [t, t + line_length*pose[:3,2]]
    return line_n, line_o, line_a

In [7]:
jointpos = p.getJointStates(0,joints)
transforms = joint_transforms(DH_Params)
frames = joint_frames(transforms)
axes = []
for joint in range(0,p.getNumJoints(0),2):
    line_n, line_o, line_a = axis_lines(transforms[joint//2+1] ,1)
    axes.append(p.addUserDebugLine(line_n[0], line_n[1], parentObjectUniqueId=0,parentLinkIndex=joint,lineColorRGB=(1,0,0), lineWidth = 2))
    axes.append(p.addUserDebugLine(line_a[0], line_a[1], parentObjectUniqueId=0,parentLinkIndex=joint,lineColorRGB=(0,0,1), lineWidth = 2))

In [8]:
timeStep=1./60.
p.setTimeStep(timeStep)
p.setGravity(0,-9.8,0)
 
while (1):
	p.stepSimulation()
	targets = [(np.deg2rad(p.readUserDebugParameter(parameter)) if revolute else p.readUserDebugParameter(parameter)) for revolute,parameter in parameters]
	p.setJointMotorControlArray(0,joints,p.POSITION_CONTROL,targets)
	time.sleep(timeStep)

error: Not connected to physics server.

In [10]:
jointpos = p.getJointStates(0,joints)
transforms = joint_transforms(DH_Params)
frames = joint_frames(transforms)
axes=[]
i=0
for frame in frames:
    line_n, line_o, line_a = axis_lines(frame,1)
    axes.append(p.addUserDebugLine(line_n[0], line_n[1], parentObjectUniqueId=0,lineColorRGB=(1,0,0), lineWidth = 2))
    axes.append(p.addUserDebugLine(line_a[0], line_a[1], parentObjectUniqueId=0,lineColorRGB=(0,0,1), lineWidth = 2))
    i+=2

In [11]:
timeStep=1./60.
p.setTimeStep(timeStep)
p.setGravity(0,-9.8,0)
 
while (1):
	p.stepSimulation()
	targets = [(np.deg2rad(p.readUserDebugParameter(parameter)) if revolute else p.readUserDebugParameter(parameter)) for revolute,parameter in parameters]
	p.setJointMotorControlArray(0,joints,p.POSITION_CONTROL,targets)
	jointpos = p.getJointStates(0,joints)
	frames = joint_frames(joint_transforms(DH_Params))
	i=0
	for frame in frames:
		line_n, line_o, line_a = axis_lines(frame,1)
		p.addUserDebugLine(line_n[0], line_n[1], parentObjectUniqueId=0,lineColorRGB=(1,0,0), lineWidth = 2,replaceItemUniqueId = axes[i])
		p.addUserDebugLine(line_a[0], line_a[1], parentObjectUniqueId=0,lineColorRGB=(0,0,1), lineWidth = 2,replaceItemUniqueId = axes[i+1])
		i+=2
	time.sleep(timeStep)

error: Not connected to physics server.

In [ ]:
jointpos = p.getJointStates(0,joints)
transforms = joint_transforms(DH_Params)
frames = joint_frames(transforms)
axes = []
for joint in range(0,p.getNumJoints(0),2):
    rotm = np.copy(frames[joint//2])
    rotm[0,3] = rotm[1,3] = rotm[2,3] = 0
    line_n, line_o, line_a = axis_lines(rotm.dot(transforms[joint//2+1]) ,1)
    print( joint)
    axes.append(p.addUserDebugLine(line_n[0], line_n[1], parentObjectUniqueId=0,parentLinkIndex=joint,lineColorRGB=(1,0,0), lineWidth = 2))
    axes.append(p.addUserDebugLine(line_a[0], line_a[1], parentObjectUniqueId=0,parentLinkIndex=joint,lineColorRGB=(0,0,1), lineWidth = 2))